### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

CPD_DataDirectory = os.path.join(getDataDir(), "CPD_repair_xrlesionfinder_formatted")
CPD_BaseDataFilePath = os.path.join(CPD_DataDirectory, "NHF1_CPD_1h_all_reps_C_to_T_mismatches_by_read_relation_formatted_TGG_filtered.bed")

### Parse the damage position data for mutperiod input

In [ ]:
from mutperiodpy.input_parsing.ParseStandardBed import parseStandardBed

CPD_MutperiodInputFilePaths = parseStandardBed([CPD_BaseDataFilePath], hg19FastaFilePath)

### Run the mutperiod pipeline to determine translational and rotational periodicity of the data.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite
nucleosomeMapFilePaths = list()
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_hybrid_nuc_map", "hg19_hybrid_nuc_map.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map", "hg19_LCL_MNase_nuc_map.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_euchromatin",
                                           "hg19_LCL_MNase_nuc_map_euchromatin.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_heterochromatin",
                                           "hg19_LCL_MNase_nuc_map_heterochromatin.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_stringent_euchromatin",
                                           "hg19_LCL_MNase_nuc_map_stringent_euchromatin.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_NHF1_MNase_nuc_map", "hg19_NHF1_MNase_nuc_map.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_CTCF_known", "hg19_CTCF_known.bed"))

nucleosomeMapNames = [os.path.basename(nucleosomeMapFilePath).rsplit('.', 1)[0] for nucleosomeMapFilePath in nucleosomeMapFilePaths]

runAnalysisSuite(CPD_MutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "No Normalization", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)
runAnalysisSuite(CPD_MutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Singlenuc/Dinuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)
runAnalysisSuite(CPD_MutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Trinuc/Quadrunuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)

In [ ]:
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr
from mutperiodpy.RunNucleosomeMutationAnalysis import runNucleosomeMutationAnalysis
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory

runNucleosomeMutationAnalysis(getFilesInDirectory(CPD_DataDirectory, DataTypeStr.rawNucCounts + ".tsv"),
                              outputFilePath = os.path.join(CPD_DataDirectory, "CPD_repair_control_raw_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = True)
runNucleosomeMutationAnalysis(getFilesInDirectory(CPD_DataDirectory, DataTypeStr.normNucCounts + ".tsv"),
                              outputFilePath = os.path.join(CPD_DataDirectory, "CPD_repair_control_normalized_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = True)